In [1]:
import pandas as pd
import numpy as np
import pickle
from tqdm import tqdm
import seaborn as sns
import matplotlib.pyplot as plt
from collections import defaultdict
from sklearn import metrics
import numpy as np
from tqdm import tqdm
from sklearn.metrics import precision_recall_fscore_support
import glob

sns.set(style='white', context='notebook', rc={'figure.figsize': (14, 10)})
from tqdm import tqdm

In [4]:
model2outs = {}
for fname in [l for l in glob.glob("ml_out2/ml_outs*.pkl") if "14" in l or "3" in l]:
    with open(fname, "rb") as f:
        ml_outs = pickle.load(f)
    config_name = f"{ml_outs['name']}${ml_outs['embedding_mode']}${ml_outs['wiki']}${ml_outs['pca']}"
    y = ml_outs['y']
    model_y_pred = ml_outs['model_y_pred']
    model_y_proba = ml_outs['model_y_proba']
    success_labels = ml_outs['success_bin']
    model2outs[config_name] = (success_labels, y, model_y_pred, model_y_proba)

In [3]:
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import accuracy_score
POS_LABEL = 1
dset_size = 450

In [5]:
n_bootstraps = 1000

model2metrics_season_all = {}
for model in tqdm(model2outs):
    model2metrics_season = {}
    (season_labels, y, model_y_pred, _) = model2outs[model]
    my_season_labels = np.array([s.split("_")[1] for s in season_labels])
    for season in np.unique(my_season_labels):
        for _ in range(n_bootstraps):
            if season not in model2metrics_season:
                model2metrics_season[season] = []
            season_idx = np.where(my_season_labels == season)[0]
            season_idx = season_idx[season_idx >= dset_size]
            try:
                assert len(season_idx) > 0
            except:
                print(season)
                assert 1==2
            sampled_idx = np.random.choice(season_idx,
                                        size=season_idx.size, replace=True)
            acc = accuracy_score(y[sampled_idx], model_y_pred[sampled_idx])
            model2metrics_season[season].append((acc, len(season_idx)))
    model2metrics_season_all[model] = model2metrics_season

100%|██████████| 18/18 [00:14<00:00,  1.27it/s]


In [7]:
my_dic = {'season': [], 'score': [], 'mean': [], 'model_name': [],
          'median': [], 'lower': [], 'upper': [], 'sd': [], 'config': [],
          'support': []}

def model2name(x1, x2, x3):
    if x1 == "binary" and x3 == "False":
        return "binary-base"
    elif x1 == "binary" and x3 == "True":
        return "binary-pca"
    elif x1 == "entity" and x2 == "late" and x3 == "True":
        return "late-pca"
    elif x1 == "entity" and x2 == "late" and x3 == "False":
        return "late-base"
    elif x1 == "entity" and x2 == "hist" and x3 == "True":
        return "hist-pca"
    elif x1 == "entity" and x2 == "hist" and x3 == "False":
        return "hist-base"
    else:
        return ValueError("ok")

for model2metrics_name in model2metrics_season_all:
    model2metrics_season = model2metrics_season_all[model2metrics_name]
    for season in model2metrics_season:
        tup_list = model2metrics_season[season]
        acc_scores = [t[0] for t in tup_list]
        sups = [t[1] for t in tup_list]
        pookie = model2metrics_name.split("$")
        my_dic['model_name'].append(pookie[0])
        my_dic['config'].append(model2name(pookie[1], pookie[2], pookie[3]))
        my_dic['season'].append(season)
        my_dic['score'].append('acc')

        my_dic['mean'].append(
            np.mean(acc_scores))
        my_dic['median'].append(
            np.median(acc_scores))
        my_dic['sd'].append(
            np.std(acc_scores,ddof=1))
        #75%
        my_dic['lower'].append(
            np.quantile(acc_scores, 0.15))
        my_dic['upper'].append(
            np.quantile(acc_scores, 0.85))
        my_dic['support'].append(sups[0])

In [8]:
res_df = pd.DataFrame(my_dic)

In [9]:
res_df

,season,score,mean,model_name,median,lower,upper,sd,config,support
0,"(-0.000909, 0.303]",acc,0.962846,meta,0.961538,0.923077,1.000000,0.037444,binary-pca,26
1,"(0.303, 0.606]",acc,0.489512,meta,0.487805,0.414634,0.560976,0.075605,binary-pca,41
2,"(0.606, 0.909]",acc,0.525596,meta,0.526316,0.456140,0.596491,0.067228,binary-pca,57
3,"(-0.000909, 0.303]",acc,1.000000,meta,1.000000,1.000000,1.000000,0.000000,binary-base,26
4,"(0.303, 0.606]",acc,0.487756,meta,0.487805,0.390244,0.560976,0.078012,binary-base,41
5,"(0.606, 0.909]",acc,0.563842,meta,0.561404,0.491228,0.631579,0.067457,binary-base,57
6,"(-0.000909, 0.303]",acc,0.961346,meta,0.961538,0.923077,1.000000,0.037918,late-pca,26
7,"(0.303, 0.606]",acc,0.485293,meta,0.487805,0.414634,0.560976,0.077623,late-pca,41
8,"(0.606, 0.909]",acc,0.524211,meta,0.526316,0.456140,0.596491,0.065384,late-pca,57
9,"(-0.000909, 0.303]",acc,0.921731,cast_prod_text_meta,0.923077,0.884615,0.961538,0.051221,binary-pca,26


In [63]:
res_df.to_csv("station_timeslot_acc063024.csv")